In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
import string
from fuzzywuzzy import fuzz
from Levenshtein import ratio as levenshtein_ratio
from sklearn.metrics import jaccard_score
from collections import Counter

In [2]:
df_dataset = pd.read_excel('Dataset.xlsx')
df_dataset.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [3]:
df_dataset.shape

(83562, 4)

In [4]:
df_dataset.drop_duplicates(inplace=True)
df_dataset.shape

(49572, 4)

In [5]:
df_dataset = df_dataset.drop_duplicates()

In [6]:
df_dataset

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
7,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 1 شريط * 14 كبسولة,56.5
8,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم,56.5
9,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 كبسول س ج,56.5
...,...,...,...,...
83557,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مم ق,39.0
83558,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مل اقراص,39.0
83559,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم م.ر 30 قرص,39.0
83560,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم ام ار اقراص,39.0


In [7]:
df_dataset.to_csv('Dataset.csv', index=False, encoding='utf-8-sig')

In [8]:
!pip install fuzzywuzzy 

In [9]:
df_master = pd.read_excel('Master File.xlsx')
df_master.head()

,sku,product_name,product_name_ar,price
0,279,ANAFRONIL 75 MG 20 TAB,انافرونيل 75 مجم اس ار 20 قرص,75.0
1,2282,LOPRECOUGH SYRUP 100 ML,لوبريكاف شراب 100 مل,28.5
2,4331,TOMEX PLUS 50 TAB,تومكس بلس 50 قرص,60.0
3,1022,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم,129.0
4,116,GLIPTUS PLUS 50/1000 MG 30 TAB,جليبتس بلس 50/1000 مجم 30 قرص,192.0


In [10]:
df_master.to_csv('Master.csv', index=False, encoding='utf-8-sig')

In [11]:
!pip install Levenshtein

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def preprocess_text(text, language='arabic'):
    """Further improved text preprocessing."""
    if pd.isna(text) or not isinstance(text, str):
        return ""

    text = str(text).lower().strip()

    text = re.sub(r'[\u064B-\u065F]', '', text)

    replacements = {
        r'\b(مجم|ملغ|mg|milligram)\b': ' mg ',
        r'\b(جم|g|gram)\b': ' g ',
        r'\b(مل|ml|milliliter)\b': ' ml ',
        r'\b(ميكروجرام|mcg|microgram)\b': ' mcg ',
        r'\b(وحدة دولية|iu|international units)\b': ' iu ',
        r'\b(كبسولات|كبسولة|capsules|capsule|cap|caps)\b': ' capsule ',
        r'\b(اقراص|قرص|tablets|tablet|tab|tabs)\b': ' tablet ',
        r'\b(شراب|syrups|syrup)\b': ' syrup ',
        r'\b(مرهم|ointments|ointment)\b': ' ointment ',
        r'\b(جل|gels|gel)\b': ' gel ',
        r'\b(كريم|creams|cream)\b': ' cream ',
        r'\b(قطرات|قطرة|drops|drop)\b': ' drops ',
        r'\b(امبولات|امبولة|امبول|ampoules|ampoule|amp)\b': ' ampoule ',
        r'\b(فيال|vials|vial)\b': ' vial ',
        r'\b(شريط|شرايط|strips|strip)\b': ' strip ',
        r'\b(معلق|suspension|susp)\b': ' suspension ',
        r'\b(محلول|solution|sol)\b': ' solution ',
        r'\b(حقن|injection|inj)\b': ' injection ',
    }

    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)


    text = text.translate(str.maketrans('', '', string.punctuation.replace('%', '').replace('*', '')))

    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [14]:
def extract_dosage_form(text):
    """Improved extraction of dosage and form."""
    dosage_units_regex = r'(mg|g|ml|mcg|iu|%)'
    form_regex = r'(capsule|tablet|syrup|ointment|gel|cream|drops|ampoule|vial|strip|suspension|solution|injection)'

    dosage_match = re.search(r'(\d+\.?\d*)\s*' + dosage_units_regex, text, re.IGNORECASE)
    form_match = re.search(form_regex, text, re.IGNORECASE)

    dosage_value = dosage_match.group(1) + dosage_match.group(2) if dosage_match else None
    form_value = form_match.group(1) if form_match else None

    return dosage_value, form_value

df_dataset = pd.read_csv("Dataset.csv")
df_master = pd.read_csv("Master.csv")

df_dataset['processed_seller_name'] = df_dataset['seller_item_name'].apply(lambda x: preprocess_text(x, language='arabic'))
df_master['processed_product_name'] = df_master['product_name'].apply(lambda x: preprocess_text(x, language='english'))
df_master['processed_product_name_ar'] = df_master['product_name_ar'].apply(lambda x: preprocess_text(x, language='arabic'))

df_master['combined_master_name'] = df_master['product_name_ar'] + ' ' + df_master['product_name']

df_dataset[['seller_dosage', 'seller_form']] = df_dataset['processed_seller_name'].apply(extract_dosage_form).tolist()
df_master[['master_dosage', 'master_form']] = df_master['combined_master_name'].apply(extract_dosage_form).tolist()

vectorizer = TfidfVectorizer(ngram_range=(2, 4), max_df=0.8, min_df=2, analyzer='char_wb')
all_text = pd.concat([df_dataset['processed_seller_name'], df_master['combined_master_name']]).unique()
vectorizer.fit(all_text)

master_tfidf_matrix = vectorizer.transform(df_master['combined_master_name'])

In [15]:
def calculate_similarity(row, master_tfidf_matrix, master_df):
    """Enhanced similarity calculation with Jaccard and improved weighting, Language Specific Fuzzy & Levenshtein"""
    seller_name = row['processed_seller_name']
    seller_vec = vectorizer.transform([seller_name])

    cosine_similarities = cosine_similarity(seller_vec, master_tfidf_matrix)[0]


    is_arabic = bool(re.search('[\u0600-\u06FF]', seller_name))

    if is_arabic:
        fuzzy_scores = master_df['product_name_ar'].apply(lambda master_name_ar: fuzz.token_set_ratio(seller_name, master_name_ar) / 100.0 if isinstance(master_name_ar, str) else 0) # Handle NaN
        levenshtein_scores = master_df['product_name_ar'].apply(lambda master_name_ar: levenshtein_ratio(seller_name, master_name_ar) if isinstance(master_name_ar, str) else 0) # Handle NaN
    else:
        fuzzy_scores = master_df['product_name'].apply(lambda master_name_en: fuzz.token_set_ratio(seller_name, master_name_en) / 100.0 if isinstance(master_name_en, str) else 0) # Handle NaN
        levenshtein_scores = master_df['product_name'].apply(lambda master_name_en: levenshtein_ratio(seller_name, master_name_en) if isinstance(master_name_en, str) else 0) # Handle NaN


    seller_tokens = set(seller_name.split())
    jaccard_scores = master_df['combined_master_name'].apply(lambda master_name: len(seller_tokens & set(master_name.split())) / len(seller_tokens | set(master_name.split())) if seller_tokens else 0)

    WEIGHT_COSINE = 0.5 
    WEIGHT_FUZZY = 0.2
    WEIGHT_LEVENSHTEIN = 0.1
    WEIGHT_JACCARD = 0.1
    WEIGHT_DOSAGE = 0.05
    WEIGHT_FORM = 0.05


    combined_scores = (
        WEIGHT_COSINE * cosine_similarities +
        WEIGHT_FUZZY * fuzzy_scores +
        WEIGHT_LEVENSHTEIN * levenshtein_scores +
        WEIGHT_JACCARD * jaccard_scores
    )

    most_similar_index = np.argmax(combined_scores)
    predicted_sku = master_df['sku'].iloc[most_similar_index]


    matched_master_product_name_en = master_df[master_df['sku'] == predicted_sku]['product_name'].iloc[0] if not master_df[master_df['sku'] == predicted_sku]['product_name'].empty else 'Unknown'
    matched_master_product_name_ar = master_df[master_df['sku'] == predicted_sku]['product_name_ar'].iloc[0] if not master_df[master_df['sku'] == predicted_sku]['product_name_ar'].empty else 'Unknown'


    return predicted_sku, combined_scores[most_similar_index], matched_master_product_name_en, matched_master_product_name_ar

df_dataset[['predicted_sku', 'similarity_score', 'matched_master_product_name_en', 'matched_master_product_name_ar']] = df_dataset.apply(
    calculate_similarity, axis=1, result_type='expand', args=(master_tfidf_matrix, df_master)
)

accuracy = accuracy_score(df_dataset['sku'], df_dataset['predicted_sku'])
print(f"Improved Accuracy: {accuracy * 100:.2f}%")

Improved Accuracy: 91.52%


In [16]:
import pickle

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

with open("master_tfidf_matrix.pkl", "wb") as f:
    pickle.dump(master_tfidf_matrix, f)

df_master.to_csv("master_dataset.csv", index=False)

with open("vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

with open("master_tfidf_matrix.pkl", "rb") as f:
    master_tfidf_matrix = pickle.load(f)

df_test = pd.read_excel("Test_Products.xlsx")

df_test = df_test.rename(columns={'product_name': 'seller_item_name'})

df_test['processed_seller_name'] = df_test['seller_item_name'].apply(lambda x: preprocess_text(x, language='arabic'))

df_test[['seller_dosage', 'seller_form']] = df_test['processed_seller_name'].apply(extract_dosage_form).tolist()

df_test[['predicted_sku', 'similarity_score', 'matched_master_product_name_en', 'matched_master_product_name_ar']] = df_test.apply(
    calculate_similarity, axis=1, result_type='expand', args=(master_tfidf_matrix, df_master)
)

df_test.to_excel("Test_Results.xlsx", index=False)

print("Testing complete! Results saved to Test_Results.xlsx")

Testing complete! Results saved to Test_Results.xlsx


In [17]:
t = pd.read_excel("Test_Results.xlsx")

In [18]:
t

,seller_item_name,processed_seller_name,seller_dosage,seller_form,predicted_sku,similarity_score,matched_master_product_name_en,matched_master_product_name_ar
0,استوهالت 40 مجم 14 ك,استوهالت 40 mg 14 ك,40mg,NaN,1322,0.575550,ESTOHALT 40 MG 14 CAP,استوهالت 40 مجم 14 كبسول
1,استوهالت 40 مجم 1 شريط * 14 كبسولة,استوهالت 40 mg 1 strip * 14 capsule,40mg,strip,1322,0.480737,ESTOHALT 40 MG 14 CAP,استوهالت 40 مجم 14 كبسول
2,استوهالت 40 مجم,استوهالت 40 mg,40mg,NaN,1322,0.524712,ESTOHALT 40 MG 14 CAP,استوهالت 40 مجم 14 كبسول
3,استوهالت 40 كبسول س ج,استوهالت 40 كبسول س ج,NaN,NaN,1322,0.558107,ESTOHALT 40 MG 14 CAP,استوهالت 40 مجم 14 كبسول
4,Augmentin 10mg,augmentin 10mg,10mg,NaN,8,0.461982,AUGMENTIN 1 G 14 TAB,اوجمنتين 1 جم 14 قرص
5,TAROLIMUS 0.03% OINT. 15 GM,tarolimus 003% oint 15 gm,003%,NaN,1022,0.540437,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم
6,DALACIN C 300 MG 10 CAP,dalacin c 300 mg 10 capsule,300mg,capsule,219,0.609553,DALACIN C 300 MG 10 CAP,دالاسين سي 300 مجم 10 كبسول
7,ARTHROFREE 25 MG 30 TAB,arthrofree 25 mg 30 tablet,25mg,tablet,366,0.465842,ARTHFREE 20 MG 30 TAB,ارث فرى 20 مجم 30 قرص
8,FLESTOR 50 MG 30 CAP,flestor 50 mg 30 capsule,50mg,capsule,446,0.474203,FLECTOR 50 MG 30 CAP,فليكتور 50 مجم 30 كبسول
9,ATACAND 8 MG 14 TAB,atacand 8 mg 14 tablet,8mg,tablet,874,0.638183,ATACAND 8 MG 14 TAB,اتاكاند 8 مجم 14 قرص
